# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import pycountry_convert as pc

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vac_df = pd.read_csv('../output_data/cities.csv') 
vac_df

,City,Country,Latitude,Longitude,Max_temp_F,Humidity_Percent,Cloudy_Percent,Wind_Speed_mph,Date
0,Mataura,New Zealand,-46.19,168.86,50.94,70,39,3.29,12/15/20
1,Whitehorse,Canada,60.72,-135.05,6.80,84,90,2.24,12/15/20
2,Hualmay,Peru,-11.10,-77.61,73.31,67,43,10.40,12/15/20
3,Birjand,Iran,32.87,59.22,30.20,63,0,6.93,12/15/20
4,Albany,United States,42.60,-73.97,33.01,36,9,1.99,12/15/20
...,...,...,...,...,...,...,...,...,...
537,Hatfield,United Kingdom,51.76,-0.22,48.00,87,75,8.05,12/15/20
538,Srednekolymsk,Russia,67.45,153.68,-26.93,86,99,2.64,12/15/20
539,Ballina,Australia,-28.87,153.57,75.20,94,75,6.93,12/15/20
540,Hofn,Iceland,64.25,-15.21,39.04,92,100,22.91,12/15/20


In [3]:
vac_df.dtypes

City                 object
Country              object
Latitude            float64
Longitude           float64
Max_temp_F          float64
Humidity_Percent      int64
Cloudy_Percent        int64
Wind_Speed_mph      float64
Date                 object
dtype: object

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

In [5]:
# Store 'Lat' and 'Lng' into  locations 
locations = vac_df[["Latitude", "Longitude"]].astype(float)

# Weight
humheat = vac_df["Humidity_Percent"]

figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}

# Plot Heatmap
fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,20))

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humheat, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig



Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
#Vacation spots

vac_spot = (vac_df["Max_temp_F"] < 80) & (vac_df["Max_temp_F"] > 40) & (vac_df["Humidity_Percent"] < 65)& (vac_df["Country"] == "Chile") 

vac_spot = vac_spot.to_frame('Destination')

vac_dest = vac_df.merge(vac_spot, how = "outer", left_index=True, right_index=True)
# vac_dest
vac_dest["Destination"] = vac_dest["Destination"]*1
# vac_dest

vac_best = vac_dest.loc[vac_dest['Destination'] == 1]
vac_best

,City,Country,Latitude,Longitude,Max_temp_F,Humidity_Percent,Cloudy_Percent,Wind_Speed_mph,Date,Destination
60,Lebu,Chile,-37.62,-73.65,56.30,60,0,23.26,12/15/20,1
69,Cape,Chile,-36.41,-71.90,73.40,31,0,12.75,12/15/20,1
147,Constitución,Chile,-35.33,-72.42,67.93,36,0,18.90,12/15/20,1
222,Ancud,Chile,-41.87,-73.82,55.40,54,75,8.05,12/15/20,1
255,Coyhaique,Chile,-45.58,-72.07,51.80,53,90,8.05,12/15/20,1
256,Coihueco,Chile,-36.62,-71.83,73.40,31,0,12.75,12/15/20,1


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
hotel_df = vac_best.copy()
del hotel_df['Cloudy_Percent']

hotel_df.rename(columns={'Latitude':'Lat', 'Longitude':'Lng',}, inplace=True)
hotel_df

,City,Country,Lat,Lng,Max_temp_F,Humidity_Percent,Wind_Speed_mph,Date,Destination
60,Lebu,Chile,-37.62,-73.65,56.30,60,23.26,12/15/20,1
69,Cape,Chile,-36.41,-71.90,73.40,31,12.75,12/15/20,1
147,Constitución,Chile,-35.33,-72.42,67.93,36,18.90,12/15/20,1
222,Ancud,Chile,-41.87,-73.82,55.40,54,8.05,12/15/20,1
255,Coyhaique,Chile,-45.58,-72.07,51.80,53,8.05,12/15/20,1
256,Coihueco,Chile,-36.62,-71.83,73.40,31,12.75,12/15/20,1


In [13]:
# params dictionary to update each iteration
params = {
    "radius": 10000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
hotel_df

,City,Country,Lat,Lng,Max_temp_F,Humidity_Percent,Wind_Speed_mph,Date,Destination,Hotel Name
60,Lebu,Chile,-37.62,-73.65,56.30,60,23.26,12/15/20,1,Konui Hotel-Lodge
69,Cape,Chile,-36.41,-71.90,73.40,31,12.75,12/15/20,1,Hotel Molino Viejo
147,Constitución,Chile,-35.33,-72.42,67.93,36,18.90,12/15/20,1,Bed & Breakfast Sweet Dreams
222,Ancud,Chile,-41.87,-73.82,55.40,54,8.05,12/15/20,1,Panamericana Hotel Ancud
255,Coyhaique,Chile,-45.58,-72.07,51.80,53,8.05,12/15/20,1,Hotel Dreams de la Patagonia
256,Coihueco,Chile,-36.62,-71.83,73.40,31,12.75,12/15/20,1,Aqualipso Agroturismo


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
locations

,Lat,Lng
60,-37.62,-73.65
69,-36.41,-71.90
147,-35.33,-72.42
222,-41.87,-73.82
255,-45.58,-72.07
256,-36.62,-71.83


In [16]:
# Create hotel layer

hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=6,
    info_box_content=[f"{hotel}" for hotel in hotel_info])

figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}

fig = gmaps.figure(layout=figure_layout,zoom_level=4,center=(-40,-60))

fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

In [17]:
# Add marker layer ontop of heat map

figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}

fig = gmaps.figure(layout=figure_layout,zoom_level=4,center=(-40,-60))

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

fig

# Display figure


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…